 <h3><b>Data Prepparation</b></h3>

In [1]:
import numpy as np
import pandas as pd

In [2]:
datatrain = pd.read_csv('Combined_file.csv')
datatrain = datatrain.iloc[0:45]
datatrain

FileNotFoundError: [Errno 2] No such file or directory: 'Combined_file.csv'

In [3]:
print(type(datatrain),"Shape:", datatrain.shape)

<class 'pandas.core.frame.DataFrame'> Shape: (45, 17)


In [4]:
datatrain.drop(columns=["T0"], inplace = True)

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
#Here std deviation and mean is calculated
scaler = scaler.fit(datatrain) 
#Here normalization is done according to obtained mean and std deviation
train_scaled = np.array(scaler.transform(datatrain)) #IT WILL RETURN NUMPY.NDARRAY

In [6]:
trainX = (train_scaled[:,0:15])
trainY = (train_scaled[:,15])

In [7]:
print(trainX.shape, type(trainX))

(45, 15) <class 'numpy.ndarray'>


<h3><b>Defining Objective function</b></h3>

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

2024-03-18 21:06:11.728460: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-18 21:06:11.789572: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 21:06:11.789623: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 21:06:11.789670: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-18 21:06:11.800215: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-18 21:06:13.377309: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT

In [9]:
from tensorflow.keras import regularizers

In [10]:
n_past = 10

In [11]:
#Making tensor data
ftrainX = []
m = trainX.shape[0]

for k in range(m - n_past):
    ftrainX.append(trainX[k:k+n_past, :])
ftrainY = trainY[n_past:]
#Converting to numpy array
ftrainX,ftrainY = np.array(ftrainX),np.array(ftrainY)

#Train_test_split
X_train, X_test,Y_train, Y_test = train_test_split(ftrainX, ftrainY, test_size = 0.3)

print("trainX: ",ftrainX.shape, type(ftrainX))
print("trainY: ",ftrainY.shape,type(ftrainY))
print("X_test: ",X_test.shape,type(X_test))
print("Y_test: ",Y_test.shape,type(Y_test))

trainX:  (35, 10, 15) <class 'numpy.ndarray'>
trainY:  (35,) <class 'numpy.ndarray'>
X_test:  (11, 10, 15) <class 'numpy.ndarray'>
Y_test:  (11,) <class 'numpy.ndarray'>


In [2]:
# Define the objective function (MSE in this case)
def objective_function(parameters):
    # Set the LSTM parameters
    lstm_units = int(parameters[0])
    adam_init = parameters[2]
    epoch = int(parameters[3])
    
    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(lstm_units, input_shape=(n_past, 15), return_sequences = True, kernel_regularizer = regularizers.L2(0.01)))
    model.add(Dense(1))
    model.compile(loss = 'mse', optimizer = Adam(adam_init))

    # Train the LSTM model
   
    model.fit(X_train, Y_train, epochs=epoch, validation_split = 0.3, batch_size=32, verbose=0) #verbose = 1, to display each error in each iteration

    # Make predictions using the trained model
    y_pred = model.predict(X_test)

    # Calculate the mean squared error (MSE)
    mse = mean_squared_error(Y_test, y_pred)
    return mse

<h3><b>GWO Algorithm</b></h3>

In [11]:
#Definig minimum function

def my_min(A):       
    """
    A should be list or array
    It returns index of least value in the array
    """
    mini = A[0]
    index = 0
    for i in range(len(A)):
        if A[i] < mini:
            mini = A[i]
            index = i
    return index

def get_pos(Arry):
    """
    It takes an array and return index of least 3 values.
    Since it is set to 3, it will return 3 indice. we can change this value also
    """
    pos = []
    for i in range(3):
        pos.append(my_min(Arry))
        Arry[pos[-1]] = float('inf')
    return pos


#get_pos([2,13,4,6,5])

In [12]:
# Lower bounds for LSTM parameters [ layer1 units, layer2 units, adam_init, epoch]
# Upper bounds for LSTM parameters [ LSTM units, LSTM2 units, adam_init, epoch]
def check_bound(new_pos, l, u):
    for i in range(len(new_pos)):
        if new_pos[i]<l[i]:
            new_pos[i] = l[i]
        elif new_pos[i]>u[i]:
            new_pos[i] = u[i]
    return new_pos

In [18]:
# Gray Wolf Optimizer implementation
def gwo_optimizer(objective_function, lb: np.array, ub: np.array, dimension, population_size: int , iterations: int):
    """
    objective_function: function to be optimized
    lb: list of lowest values of parameters, its length depends on no.of parameters to be optimized
    ub: list of highest values of parameters, its length depends on no.of parameters and same as lb
    dimension: no of variabes to be optimized = len(lb  or ub)
    population_size: It is number of wolves, that will help in optimization
    iteration: maximum number of iteration to be performed for geting optimum values of parameters
    """
    
    # Initialize the position of the wolves
    positions = np.zeros((population_size, dimension))
    for i in range(population_size):
        positions[i] = lb + (ub - lb)*np.random.rand(dimension)
    
    #print(positions)
      
    #Initializing fitness of the wolves
    fitness = np.zeros(population_size)
    for i in range(population_size):
        fitness[i] = objective_function(positions[i])

    # Optimization loop
    for t in range(iterations):
        # Initialize the best positions of the wolves
        alpha, beta, delta = get_pos(fitness)
        
        alpha_pos = positions[alpha]
        beta_pos = positions[beta]
        delta_pos = positions[delta]
        
        a = 2 - 2*(t / iterations)  # a decreases linearly in [2:0)
        r = np.random.rand(2,3)  
        A = np.matmul(np.array([[2*a, 0]]), r) - np.array([[a, a, a]])   # [[A1, A2, A3]]
        C = np.matmul(np.array([[0, 2]]) , r)  #[[C1, C2, C3]]

        # Update the positions of the wolves
        for j in range(population_size):
            #print('POSITION: ', j)
            
            # D = [[D_alpha, D_beta, D_delta]]
            D = np.array([C[0,0]*alpha_pos, C[0,1]*beta_pos, C[0,2]*delta_pos]) - np.array([positions[j], positions[j], positions[j]])
            
            X = np.array([alpha_pos, beta_pos, delta_pos])-np.array([A[0,0]*D[0], A[0,1]*D[1], A[0,2]*D[2]])
            X_new = np.average(X, axis=0)
            
            new_pos = check_bound(X_new,lb,ub)  # mean position of updated alpha, beta & delta

            fit_score = objective_function(new_pos)
            if fitness[j]> fit_score:
                positions[j] = new_pos
                fitness[j] = fit_score
                
        print(f'iteration: {t+1}, \n alpha = {alpha_pos},\n beta = {beta_pos},\n delta = {delta_pos}\n','New pos:', new_pos)

    # Return the best solution found
    best_solution = alpha_pos
    return best_solution

In [6]:
import numpy as np

r = np.random.rand(2,3) 
a = 1.5
A = np.matmul(np.array([[2*a, 0]]), r) - np.array([[a, a, a]])
A

array([[-1.19000922, -0.08222195, -0.96372867]])

In [2]:
def Himmelblau(inp):
    """
    x: [-5,5]
    y: [-5,5]
    optimum value: [3.0, 2.0], [-2.805118, 3.131312], [-3.779310, -3.283186], [3.584428, -1.848126]
    """
    x,y = inp[0], inp[1]
    return (x**2 + y - 11)**2 + (x + y**2 -7)**2

In [19]:
lb = [-5,-5]
ub = [5,5]

population_size = 5
Max_iter = 1000
dim = len(ub)

optimized_params = gwo_optimizer(Himmelblau, np.array(lb), np.array(ub), dim, population_size, Max_iter)


iteration: 1, 
 alpha = [-2.72981218 -0.12655056],
 beta = [-4.02649358 -1.56370176],
 delta = [2.42950941 2.02449913]
 New pos: [2.07180436 0.74217127]
iteration: 2, 
 alpha = [3.51151566 1.71331026],
 beta = [3.03248431 1.65669951],
 delta = [3.57153054 1.46223052]
 New pos: [3.57153054 1.46223052]
iteration: 3, 
 alpha = [4.86943966 2.11315416],
 beta = [5.         2.23799519],
 delta = [5.         2.47219432]
 New pos: [5.         2.47219432]
iteration: 4, 
 alpha = [0.86574335 0.67447317],
 beta = [-0.06707966  0.24091195],
 delta = [-1.0765807  -0.16121513]
 New pos: [0.61080505 0.46962898]
iteration: 5, 
 alpha = [1.13098419 0.88560098],
 beta = [1.5784174  1.23648072],
 delta = [2.55198794 2.00331186]
 New pos: [2.55198794 2.00331186]
iteration: 6, 
 alpha = [2.4091553  1.86739578],
 beta = [2.4497224  1.55229918],
 delta = [1.92975133 1.32118388]
 New pos: [2.4091553  1.86739578]
iteration: 7, 
 alpha = [2.25181953 1.43075749],
 beta = [2.3358004  1.64726749],
 delta = [2.5651

In [15]:
optimized_params

array([2.49138093, 2.49138093])

In [9]:
from optimizers import GWO
ans = GWO(Himmelblau, lb, ub, dim, 5, 1000)
print(ans)

GWO is optimizing  "Himmelblau"
['At iteration 0 the best fitness is 44.43034785591318']
['At iteration 1 the best fitness is 44.43034785591318']
['At iteration 2 the best fitness is 44.43034785591318']
['At iteration 3 the best fitness is 15.65386065061635']
['At iteration 4 the best fitness is 7.819670230525257']
['At iteration 5 the best fitness is 3.424792612678644']
['At iteration 6 the best fitness is 3.424792612678644']
['At iteration 7 the best fitness is 3.424792612678644']
['At iteration 8 the best fitness is 3.424792612678644']
['At iteration 9 the best fitness is 3.424792612678644']
['At iteration 10 the best fitness is 3.424792612678644']
['At iteration 11 the best fitness is 3.424792612678644']
['At iteration 12 the best fitness is 3.424792612678644']
['At iteration 13 the best fitness is 3.424792612678644']
['At iteration 14 the best fitness is 3.424792612678644']
['At iteration 15 the best fitness is 3.424792612678644']
['At iteration 16 the best fitness is 3.4247926126

In [3]:
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'